<a href="https://colab.research.google.com/github/docuracy/desCartes/blob/main/experiments/segformer-CPU-testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Authenticate GCS, mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!gcloud auth application-default login
!gcloud config set project descartes-404713

Mounted at /content/drive
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=WFW0ubDUwwpDFtDJdj4AK4QN5hmnoW&prompt=consent&token_usage=remote&access_type=offline&code_challenge=5w54helBhFCZ46oNexOqK8kksTny1WlW0l_hCvPfCX0&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQSTgQGP--7lMW6Pj7FgEYnQ8b9fhLAp28JwzK00S8lwTWVDUY9jm5ZuA2pNfZ7Bp5H35A

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Def

In [2]:
!pip install opencv-python

In [3]:
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [ ]:
# @title Setup and Imports { display-mode: "code" }
# Import necessary libraries
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor, TrainingArguments, Trainer
from google.cloud import storage

# Google Drive Path Configuration
project_path = '/content/drive/MyDrive/desCartes'
model_path = f'{project_path}/models'

# GCS Configuration
gcs_key_path = f'{project_path}/descartes-404713-cccf7c3921aa.json'
gcs_project_id = 'descartes-404713'
gcs_bucket_name = 'descartes'
gcs_data_directory = "training_data"

# Training Configuration
batch_size = 1  # Batch size for training

###################################################

# SegFormer Model and Feature Extractor
image_processor = SegformerImageProcessor.from_pretrained('nvidia/segformer-b4-finetuned-ade-512-512')
model = SegformerForSemanticSegmentation.from_pretrained('nvidia/segformer-b4-finetuned-ade-512-512')

# Dataset Class for PyTorch .pt Files
class SegmentationDataset(Dataset):
    def __init__(self, gcs_dir):
        self.gcs_dir = gcs_dir
        self.client = storage.Client(project=gcs_project_id)
        self.bucket = self.client.get_bucket(gcs_bucket_name)

        # List all the .pt files in the GCS directory
        self.files = [blob.name for blob in self.bucket.list_blobs(prefix=gcs_dir) if blob.name.endswith('.pt')]

        # Initialize the image processor
        self.image_processor = SegformerImageProcessor.from_pretrained('nvidia/segformer-b4-finetuned-ade-512-512')

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]

        # Download the .pt file from GCS
        blob = self.bucket.blob(file_path)
        local_path = f"/tmp/{file_path.split('/')[-1]}"
        blob.download_to_filename(local_path)

        # Load the PyTorch dataset (now contains a single image and label)
        data = torch.load(local_path)

        # Process the image and label
        inputs = self.image_processor(images=data['images'], return_tensors="pt", padding=True)
        pixel_values = inputs['pixel_values'].squeeze(0)
        label = data['labels'].squeeze()
        # Ensure labels are in the correct range
        label = label.long()

        return {
            "pixel_values": pixel_values,
            "labels": label
        }

# Create dataset instances for train and eval
train_dataset = SegmentationDataset(f"{gcs_data_directory}/train")
eval_dataset = SegmentationDataset(f"{gcs_data_directory}/eval")

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    eval_strategy="steps",           # evaluate every eval_steps
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    gradient_accumulation_steps=4,  # Simulates batch size of 4
    num_train_epochs=3,              # total number of training epochs
    save_steps=10_000,               # number of steps before saving the model
    save_total_limit=2,              # maximum number of checkpoints to save
)

# Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Pass the dataset to the trainer
    eval_dataset=eval_dataset,    # Pass the eval dataset to the trainer
)

# Start Training
trainer.train()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/deprecation.py:172: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)


config.json:   0%|          | 0.00/6.89k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/257M [00:00<?, ?B/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: docuracy (docuracy-university-of-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/utils/deprecation.py:172: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.preprocess` and were ignored: 'padding'
  return func(*args, **kwargs)


tensor([0, 1, 4])
tensor([0, 2, 3, 4])
tensor([0, 1, 2])
tensor([0, 2, 3])
tensor([0, 1, 2, 3, 4])


Step,Training Loss,Validation Loss


tensor([0, 1, 3, 4])
tensor([0, 2])
tensor([0, 1, 3])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2])
tensor([0, 2, 3, 4])
tensor([0, 1, 2, 3, 4])
tensor([0, 3, 4])
tensor([0, 2, 3, 4])
tensor([0, 1, 2, 3, 4])
tensor([0])
tensor([0, 2, 3])
tensor([0])
tensor([0, 1, 2, 3])
tensor([0, 1, 2, 3, 4])
tensor([0, 3, 4])
tensor([0])
tensor([0])
tensor([0, 2])
tensor([0, 1, 3, 4])
tensor([0, 1, 3, 4])
tensor([0, 1])
tensor([0, 1])
tensor([0, 1, 2, 3, 4])
tensor([0, 3, 4])
tensor([0, 1, 2])
tensor([0, 1, 2])
tensor([0, 3, 4])
tensor([0, 2, 3])
tensor([0, 1, 2, 3])
tensor([0, 1, 3, 4])
tensor([0, 1, 2, 3, 4])
tensor([0, 4])
tensor([0])
tensor([0, 2, 3, 4])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 3, 4])
tensor([0, 1, 2, 3, 4])
tensor([0])
tensor([0, 2, 3, 4])
tensor([0, 4])
tensor([0])
tensor([0, 2, 3])
tensor([0])
tensor([0, 1, 2, 3, 4])
tensor([0, 1, 2, 3])
tensor([0, 1])
tensor([0, 1, 2])
tensor([0, 1, 2])
tensor([0, 3, 4])
tensor([0, 2, 3, 4])
tensor([0, 1, 3])


In [ ]:
# @title Visualizing Results { display-mode: "code" }

# Function to display images and predicted masks
def plot_predictions(model, dataset, n_samples=3):
    for i, (images, labels) in enumerate(dataset.take(n_samples)):
        predictions = model(images).logits
        predictions = tf.argmax(predictions, axis=-1)

        for j in range(min(n_samples, len(images))):
            image = images[j].numpy()
            label = labels[j].numpy()
            prediction = predictions[j].numpy()

            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            axes[0].imshow(image)
            axes[0].set_title('Input Image')
            axes[1].imshow(np.argmax(label, axis=-1), cmap='viridis')
            axes[1].set_title('True Label')
            axes[2].imshow(prediction, cmap='viridis')
            axes[2].set_title('Predicted Mask')
            plt.show()

# Display some predictions
plot_predictions(model, val_dataset)


In [ ]:
# @title Evaluation Metrics { display-mode: "code" }
from sklearn.metrics import classification_report

# Function to calculate metrics for model evaluation
def evaluate_model(model, dataset):
    all_preds = []
    all_labels = []

    for images, labels in dataset.take(10):  # evaluate on first 10 batches
        predictions = model(images).logits
        preds = tf.argmax(predictions, axis=-1).numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

    # Flatten the lists for classification_report
    all_preds = np.concatenate(all_preds).flatten()
    all_labels = np.concatenate(all_labels).flatten()

    report = classification_report(all_labels, all_preds, output_dict=True)
    return report

# Print evaluation metrics
eval_report = evaluate_model(model, val_dataset)
print("Evaluation Metrics:\n", eval_report)


In [ ]:
# @title Model Saving { display-mode: "code" }
# Save the trained model
model.save_pretrained(f'{model_path}/segformer_model')
# Save the image processor
image_processor.save_pretrained(f'{model_path}/image_processor')


In [ ]:
# @title Visualizing Training Logs { display-mode: "code" }
import os

# Function to plot training logs
def plot_logs(log_dir='./logs'):
    log_files = [f for f in os.listdir(log_dir) if f.endswith('.json')]

    if len(log_files) == 0:
        print("No log files found.")
        return

    log_file = log_files[0]
    log_path = os.path.join(log_dir, log_file)
    logs = []

    with open(log_path, 'r') as f:
        logs = f.readlines()

    steps, losses = [], []
    for log in logs:
        if 'step' in log and 'loss' in log:
            step = int(log.split('step')[1].split(',')[0].strip())
            loss = float(log.split('loss')[1].split(',')[0].strip())
            steps.append(step)
            losses.append(loss)

    plt.plot(steps, losses)
    plt.xlabel('Training Steps')
    plt.ylabel('Loss')
    plt.title('Training Loss Progress')
    plt.show()

# Plot the training logs
plot_logs()
